# LangGraph: Graph-Based LLM Orchestration
**Duration:** 45 minutes
**Learning Outcomes:**
- Understand what LangGraph is and why use it
- Build your first LLM graph
- Learn core concepts: Prompts, Memory, Tools, Agents
- Combine LangChain and LangGraph

---

## Section Overview

| Start Time | Activity | Duration |
|------|----------|----------|
| 13:45 | Intro + Example | 7 mins |
| 13:52 | Memory Demo | 5 mins |
| 13:57 | Tool Demo | 8 mins |
| 14:05 | Solo Exercise | 10 mins |
| 14:15 | LAB: Multi-Agent Graph | 15 mins |

*+ Time at the end*


---
## 🕸️ What is LangGraph?

LangGraph is a framework built on top of LangChain that enables developers to design stateful, multi-step, and multi-agent workflows using graph-based architectures.

Instead of linear chains, LangGraph allows you to define:
- **Nodes** → steps, agents, or tool calls
- **Edges** → transitions between steps

This enables more flexible, reliable, and production-ready AI systems.

In this module, we explore how graph-based orchestration improves reasoning, tool usage, memory persistence, and multi-agent collaboration.


---
## 🤷‍♂️ Why Use LangGraph?

LangGraph is ideal when applications require:

- Long-running workflows
- Memory persistence across steps
- Tool invocation and dynamic decision-making
- Multi-agent collaboration
- Hybrid deterministic + agentic control

It provides better state control, traceability, and reliability compared to traditional linear pipelines.


---

### 🏗️ Graph Structure
![alt text](image.png)

---

### 👋🌍 Hello World Example (Learning Graphing)


We import our libraries

In [2]:
# LangGraph uses dictionaries, more accurately the class TypedDict, to model state
# StateGraph is the class used to define the graph

from typing import TypedDict
from langgraph.graph import StateGraph

We define our Agent state and node functionality

In [3]:
#You define a child class of TypedDict to be the state of your graph

class AgentState(TypedDict):
    message : str
    name: str

#Nodes are functions that take in state and return state
def hello_world(state: AgentState) -> AgentState:
    state['message'] = "Hello " + state['message']
    return state

def hello_name(state: AgentState) -> AgentState:
    state['message'] = state['message'] + ", Hello " + state['name']
    return state

We make the graph and compile it

In [4]:
#Here we define our graph
graph = StateGraph(AgentState)

#Nodes have to be added before being linked
graph.add_node("hello_node_1", hello_world)
graph.add_node("hello_node_2", hello_name)

graph.set_entry_point("hello_node_1") # "graph.add_edge(START, hello_node)" also works (have to import START, END)
graph.add_edge("hello_node_1", "hello_node_2")
graph.set_finish_point("hello_node_2") # "graph.add_edge(hello_node, END)" also works

app = graph.compile()

Finally, we run it, inputting an initial state.

In [5]:
app.invoke({"message": "World", "name": "<Your Name>"})['message']

'Hello World, Hello <Your Name>'

---

### 👨‍💻 Demo - Chatbot (Implementing Memory)

In [6]:
#List, Union used to define structures for memory
from typing import TypedDict, List, Union 
from langchain_core.messages import HumanMessage, AIMessage


from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
### Follow Along ###

# We first define our AgentState
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

# Then our llm object
llm = ChatOpenAI(model="gpt-4o")

# Then our nodes
def process(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    response = llm.invoke(state["messages"])

    state["messages"].append(AIMessage(content=response.content)) 
    print(f"\nAI: {response.content}")
    print("CURRENT STATE: ", state["messages"])

    return state


# We define our graph
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END) 
agent = graph.compile()


#Create a loop for repeating conversation
conversation_history = []

user_input = input("Enter: ")
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    conversation_history = result["messages"]
    user_input = input("Enter: ")


AI: Hello Kevin! How can I assist you today?
CURRENT STATE:  [HumanMessage(content='Hi my name is kevin', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Kevin! How can I assist you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


### 👨‍💻 Demo - Upgrading the Chatbot and Adding a Weather Tool
(Copy/paste the above cell or keep working on the same one)


In [12]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage # The foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage # Passes data back to LLM after it calls a tool such as the content and the tool_call_id
from langchain_core.messages import SystemMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

In [ ]:
### Follow Along ###

# We first define our AgentState
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

weather_data = {
    "London": "Rainy, 12°C",
    "New York": "Sunny, 22°C",
    "Tokyo": "Cloudy, 18°C",
}


@tool
def weather_tool(city: str):
    """This node handles the mock weather lookup"""

    # Lookup weather
    weather = weather_data.get(city, "uknown (City not found in database)")

    response_text = f"The weather in {city} is {weather}"

    print(f"\nAI (Weather Tool): {response_text}")

    return weather

tools = [weather_tool]

llm = ChatOpenAI(model = "gpt-4o").bind_tools(tools)

# Then our nodes
def agent(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    system_prompt = SystemMessage(content=
        "You are my AI assitant. Please answer my query to the best of your ability. If I make several, answer each one."
    )

    response = llm.invoke([system_prompt] + state["messages"])
    print("CURRENT STATE: ", response)
    print(f"\nAI: {response.content}")

    return {"messages": [response]}


def should_continue(state: AgentState): 
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls: 
        return "end"
    else:
        return "continue"
    

# We define our graph
graph = StateGraph(AgentState)
graph.add_node("agent", agent)
tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.add_edge(START, "agent")
graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "end": END
    }
    )

graph.add_edge("tools", "agent")

app = graph.compile()

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "First, tell me a joke. Then, tell me what's the weather in Warsaw?")]}
print_stream(app.stream(inputs, stream_mode="values"))


---

### 🌪️ Exercise - Create a Whimsical Chatbot with Calculation Tools: (Addition, Subtraction, Multiplication, Division) 

In [10]:
# Define your Agent State
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

#Define your tools:
@tool
def add(a: int, b:int):
    """This is an addition function that adds 2 numbers together"""

    return a + b 

@tool
def subtract(a: int, b: int):
    """Subtraction function"""
    return a - b

@tool
def multiply(a: int, b: int):
    """Multiplication function"""
    return a * b

@tool
def divide(a: int, b: int):
    """Division function"""
    return a / b

tools = [add, subtract, multiply, divide]

#Create your LLM object - remember to bind tools!
model = ChatOpenAI(model = "gpt-4o").bind_tools(tools)

#Create your agent node
def agent_node(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(content=
        "You are my AI assistant, please answer my query to the best of your ability."
    )
    response = model.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}

#Create your condition node
def should_continue(state: AgentState): 
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls: 
        return "end"
    else:
        return "continue"
    
#Define your graph and compile it
graph = StateGraph(AgentState)
graph.add_node("our_agent", agent_node)


tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)

graph.add_edge("tools", "our_agent")

app = graph.compile()

#The print_stream function is provided for clean serial output
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Add 40 + 12 and then multiply the result by 6. Then divide by 15. Also tell me a joke please.")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40 + 12 and then multiply the result by 6. Then divide by 15. Also tell me a joke please.
================================== Ai Message ==================================
Tool Calls:
  add (call_J43vQexOqKG8uFgooUO0OZyf)
 Call ID: call_J43vQexOqKG8uFgooUO0OZyf
  Args:
    a: 40
    b: 12
  multiply (call_7UkygYC0CP9PCdURU7YkseQm)
 Call ID: call_7UkygYC0CP9PCdURU7YkseQm
  Args:
    a: 52
    b: 6
  divide (call_YMi3EZQnv85psEtJ4b7RsI6i)
 Call ID: call_YMi3EZQnv85psEtJ4b7RsI6i
  Args:
    a: 312
    b: 15
================================= Tool Message =================================
Name: divide

20.8
================================== Ai Message ==================================

The result of \( (40 + 12) \times 6 \div 15 \) is 20.8.

And here's a joke for you:
Why don't skeletons fight each other? They don't have the guts!


### Success Criteria:
- Your agent answers prompts in a whimsical manner
- Your agent uses tools 
- Every single one of your tools works correctly
- Your agent can use tools and answer different queries in one message

---

### 🧪 LAB: Creating a Multi-Agent Graph 
Your final task to do in the remainder of this course + during some extra time at the end or back at home. Program a full LangGraph graph implementing LCEL from LangChain to create a multi-agnet system. 

You are creating a two part joke teller, where the first node would tell the set up, and the following would finish off with a punchline. You will do this task without a prior template or direct tutorial, but to balance to difficulty it is a relatively simple task.

Throughout this course you have learned how to create most of this. You just have to fit in the LCEL into each node

In [15]:
from typing import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END


In [ ]:
# Define State
class GraphState(TypedDict):
    topic: str
    joke_setup: str
    joke_punchline: str


#Create LLM

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Node 1 (Generate Joke Setup)

setup_prompt = ChatPromptTemplate.from_template(
    "Write a short setup line for a joke about: {topic}"
)

setup_chain = (
    setup_prompt
    | llm
    | StrOutputParser()
)

def generate_setup(state: GraphState):
    setup = setup_chain.invoke({"topic": state["topic"]})
    return {"joke_setup": setup}


# Node 2 = Generate Punchline

punchline_prompt = ChatPromptTemplate.from_template(
    "Given this joke setup:\n\n{setup}\n\nWrite a funny punchline."
)

punchline_chain = (
    punchline_prompt
    | llm
    | StrOutputParser()
)

def generate_punchline(state: GraphState):
    punchline = punchline_chain.invoke({"setup": state["joke_setup"]})
    return {"joke_punchline": punchline}


# Build Graph

multi_agent_graph = StateGraph(GraphState)

multi_agent_graph.add_node("setup_node", generate_setup)
multi_agent_graph.add_node("punchline_node", generate_punchline)

multi_agent_graph.set_entry_point("setup_node")

multi_agent_graph.add_edge("setup_node", "punchline_node")
multi_agent_graph.add_edge("punchline_node", END)

graph = multi_agent_graph.compile()


# Run the Graph

result = graph.invoke({"topic": "programmers"})

print("\nJoke Setup:")
print(result["joke_setup"])

print("\nPunchline:")
print(result["joke_punchline"])


Joke Setup:
Why did the programmer go broke?

Punchline:
Because he kept losing his cache!


---
## 🏁 Conclusion

LangGraph provides a powerful way to structure intelligent systems using graph-based workflows.

By combining deterministic control with agent-based reasoning, it enables scalable and production-ready AI applications.

Through learning graph modeling, tool integration, persistent memory handling, and multi-agent coordination, developers gain the foundation needed to build complex real-world AI systems.

**You now have the conceptual foundation needed to begin building LangGraph-powered systems.**




Further Learning Areas:
- Injectable states in LangGraph
- Stateful Iteration & Feedback Loops
